<a href="https://colab.research.google.com/github/florence26/Best-README-Template/blob/master/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch transformers trl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
# imports
import torch
from transformers import AutoTokenizer
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model
from trl.core import respond_to_batch

# get models
model = AutoModelForCausalLMWithValueHead.from_pretrained('gpt2')
model_ref = create_reference_model(model)

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# initialize trainer
ppo_config = PPOConfig(
    batch_size=1,
)

# encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt")

# get model response
response_tensor  = respond_to_batch(model, query_tensor)

# create a ppo trainer
ppo_trainer = PPOTrainer(ppo_config, model, model_ref, tokenizer)

# define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0)]

# train model for one step with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [4]:
train_stats

{'objective/kl': 0.0,
 'objective/kl_dist': 0.0,
 'objective/logprobs': array([[ -8.146128  ,  -2.2490625 ,  -2.660843  ,  -0.63041306,
          -1.6609842 ,  -9.227812  ,  -1.850388  ,  -0.7273568 ,
          -7.8774214 ,  -4.2604723 ,  -0.91693497,  -3.4869213 ,
          -7.496439  , -12.244268  ,  -8.733838  , -14.751516  ,
          -0.43814293,  -0.7232631 ,  -8.390723  ,  -0.4594798 ,
          -6.8647447 ,  -4.817911  ,  -2.2436304 ,  -6.5882416 ,
          -0.23531908,  -3.5011754 ]], dtype=float32),
 'objective/ref_logprobs': array([[ -8.146128  ,  -2.2490625 ,  -2.660843  ,  -0.63041306,
          -1.6609842 ,  -9.227812  ,  -1.850388  ,  -0.7273568 ,
          -7.8774214 ,  -4.2604723 ,  -0.91693497,  -3.4869213 ,
          -7.496439  , -12.244268  ,  -8.733838  , -14.751516  ,
          -0.43814293,  -0.7232631 ,  -8.390723  ,  -0.4594798 ,
          -6.8647447 ,  -4.817911  ,  -2.2436304 ,  -6.5882416 ,
          -0.23531908,  -3.5011754 ]], dtype=float32),
 'objective/k